In [ ]:
from trl import SFTTrainer, setup_chat_format
import transformers
from transformers import AutoModelForCausalLM, set_seed

In [ ]:
### Dataset processing
from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
COLUMNS_TO_KEEP = ["messages", "chosen", "rejected", "prompt", "completion", "label", "question", "answer"]

In [ ]:
def mix_datasets(dataset_mixer, shuffle=True, seed=42, test_percentage=0.1):
    '''
    Example format for dataset_nmixer:
    dataset_mixer = {
            "dataset1": 1, # dataset_name: proportion
            # "dataset1": 0.3,
            # "dataset1": 0.2,
                }
    '''
    raw_train_datasets = []
    raw_test_datasets = []
    new_dataset = DatasetDict()
    for key, value in dataset_mixer.items():
        dataset = load_dataset(key)
        if "train" in dataset:
            train_dataset = dataset["train"]
            train_dataset = train_dataset.remove_columns([col for col in train_dataset.column_names if col not in COLUMNS_TO_KEEP])
            raw_train_datasets.append((train_dataset, value))

        if "test" in dataset:
            test_dataset = dataset["test"]
            test_dataset = test_dataset.remove_columns([col for col in test_dataset.column_names if col not in COLUMNS_TO_KEEP])
            raw_test_datasets.append(test_dataset)
    train_subsets = []
    for (dataset, frac) in raw_train_datasets:
        train_subset = dataset.select(range(int(len(dataset)*frac)))
        train_subsets.append(train_subset)
    if shuffle:
        train_dataset = concatenate_datasets(train_subsets).shuffle(seed=seed)
    else:
        train_dataset = concatenate_datasets(train_subsets)
    if len(raw_test_datasets) > 0:
        test_dataset = concatenate_datasets(raw_test_datasets).shuffle(seed=seed)
    else:
        test_dataset = None
    
    new_dataset['train'] = train_dataset

    if test_dataset is None:
        new_dataset = new_dataset['train'].train_test_split(test_size=test_percentage)
    else:
        new_dataset['test'] = test_dataset
    
    return new_dataset


In [ ]:
def apply_template(example, tokenizer, task="sft"):
    if task == "sft" or task == "generation":
        if "messages" in example:
            if example["messages"][0]["role"] == "system":
                messages = example["messages"]
            else:
                messages = [{"role":"system", "content": ""}] + example["messages"]
        elif "question" in example and "answer" in example:
            if "system_prompt" in example:
                messages = [{"role":"system", "content": example['system_prompt']},{"role": "user", "content": example["question"]}, {"role": "assistant", "content": example["answer"]}]
            else:
                messages = [{"role":"system", "content": ""},{"role": "user", "content": example["question"]}, {"role": "assistant", "content": example["answer"]}]
        example['text'] = tokenizer.apply_chat_template(messages, tokenizer=False, add_generation_prompt=True if task == "generation" else False)

    elif task == "rm":
            
        


        
        


In [ ]:
dataset_mixer = {
            "dataset1": 1, # dataset_name: proportion
            # "dataset1": 0.3,
            # "dataset1": 0.2,
                }

